In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

The history saving thread hit an unexpected error (OperationalError('attempt to write a readonly database')).History will not be written to the database.
/kaggle/input/global-terrorism-database-19702020-raw/gtd_raw_1970_2020.xlsx


# Global Terrorism Database (GTD)
## Data Normalization and Modeling

**Project:** Global Terrorism Trends & Operational Dynamics (1970–2020)

### Objective
This notebook focuses on transforming the raw Global Terrorism Database (GTD)
into a structured, analysis-ready data model.

The primary goal is to normalize the raw dataset into:
- Fact tables
- Dimension tables
- Bridge tables

This modeling approach supports:
- Clean aggregation
- Accurate counting
- Efficient Tableau analysis

### Scope
- Load and inspect raw GTD data
- Validate event-level uniqueness
- Normalize data into analytical tables
- Export modeled datasets for downstream cleaning and visualization

> Note: Detailed data cleaning and validation of these tables is handled
> in a separate notebook to maintain clarity of responsibilities.


## 1. Raw Data Loading

In [2]:
GTD = "/kaggle/input/global-terrorism-database-19702020-raw/gtd_raw_1970_2020.xlsx"

In [3]:
df_gtd = pd.read_excel(GTD)

## 2. Initial Structure Inspection


In [4]:
print(df_gtd.shape)

(209706, 135)


In [5]:
df_gtd.columns.tolist()


['eventid',
 'iyear',
 'imonth',
 'iday',
 'approxdate',
 'extended',
 'resolution',
 'country',
 'country_txt',
 'region',
 'region_txt',
 'provstate',
 'city',
 'latitude',
 'longitude',
 'specificity',
 'vicinity',
 'location',
 'summary',
 'crit1',
 'crit2',
 'crit3',
 'doubtterr',
 'alternative',
 'alternative_txt',
 'multiple',
 'success',
 'suicide',
 'attacktype1',
 'attacktype1_txt',
 'attacktype2',
 'attacktype2_txt',
 'attacktype3',
 'attacktype3_txt',
 'targtype1',
 'targtype1_txt',
 'targsubtype1',
 'targsubtype1_txt',
 'corp1',
 'target1',
 'natlty1',
 'natlty1_txt',
 'targtype2',
 'targtype2_txt',
 'targsubtype2',
 'targsubtype2_txt',
 'corp2',
 'target2',
 'natlty2',
 'natlty2_txt',
 'targtype3',
 'targtype3_txt',
 'targsubtype3',
 'targsubtype3_txt',
 'corp3',
 'target3',
 'natlty3',
 'natlty3_txt',
 'gname',
 'gsubname',
 'gname2',
 'gsubname2',
 'gname3',
 'gsubname3',
 'motive',
 'guncertain1',
 'guncertain2',
 'guncertain3',
 'individual',
 'nperps',
 'nperpcap',
 

## 3. Event-Level Integrity Check

Each terrorist incident in the GTD should be uniquely identified
by an `eventid`.

This check compares total rows with unique event IDs to verify
that the dataset does not contain unintended duplication at the
incident level before normalization.


In [6]:
# Check total rows vs unique event IDs
total_rows = df_gtd.shape[0]
unique_events = df_gtd['eventid'].nunique()

print("Total rows:", total_rows)
print("Unique eventid:", unique_events)
print("Duplicate eventid count:", total_rows - unique_events)


Total rows: 209706
Unique eventid: 209706
Duplicate eventid count: 0


## 4. Rationale for Data Normalization

The raw GTD dataset contains multiple analytical concepts
(time, location, attack method, weapon type, target type, impact)
within a single wide table.

To support clean aggregation and avoid double counting,
the dataset is normalized into:
- Fact tables (event-level measures)
- Dimension tables (descriptive attributes)
- Bridge tables (many-to-many relationships)

This structure follows a star-schema–like analytical model
suitable for BI tools such as Tableau.


## 5. Identification of Core Analytical Entities

Based on analytical requirements, the following core entities
are identified:

- Incident (event-level core)
- Location
- Attack Type
- Weapon Type
- Target Type

Each entity is modeled separately to improve clarity,
reusability, and analytical accuracy.


## 6. Fact Table Construction

The fact table captures one row per terrorist incident
and contains core analytical measures and flags, including:

- Time indicators
- Incident outcome flags
- Casualty measures
- Classification indicators

This table acts as the central reference point
for all downstream analysis.


## 7. Dimension Table Construction

Dimension tables store descriptive attributes related to incidents,
such as location and categorical classifications.

Separating dimensions reduces redundancy and improves
data model interpretability.


## 8. Bridge Table Construction

Some GTD entities have many-to-many relationships with incidents.
Bridge tables are used to correctly represent these relationships
without duplicating fact records.

This ensures accurate aggregation across attack types,
weapon types, and targets.


### 6.1 Fact Table 1

In [7]:
fact_incident = df_gtd[
    [
        'eventid',
        'iyear', 'imonth', 'iday',
        'country', 'country_txt',
        'region', 'region_txt',
        'success', 'suicide',
        'doubtterr', 'multiple', 'extended'
    ]
].copy()

print(fact_incident.shape)
fact_incident.head()


(209706, 13)


,eventid,iyear,imonth,iday,country,country_txt,region,region_txt,success,suicide,doubtterr,multiple,extended
0,197000000001,1970,7,2,58,Dominican Republic,2,Central America & Caribbean,1,0,0,0.0,0
1,197000000002,1970,0,0,130,Mexico,1,North America,1,0,0,0.0,0
2,197001000001,1970,1,0,160,Philippines,5,Southeast Asia,1,0,0,0.0,0
3,197001000002,1970,1,0,78,Greece,8,Western Europe,1,0,0,0.0,0
4,197001000003,1970,1,0,101,Japan,4,East Asia,1,0,-9,0.0,0


### 7.1 Dimension Table 1

In [8]:

# Create location dimension with unique country-region combinations
dim_location = (
    fact_incident[
        ['country', 'country_txt', 'region', 'region_txt']
    ]
    .drop_duplicates()
    .reset_index(drop=True)
)

# Create surrogate key
dim_location['location_id'] = dim_location.index + 1

# Reorder columns
dim_location = dim_location[
    ['location_id', 'country', 'country_txt', 'region', 'region_txt']
]

print(dim_location.shape)
dim_location.head()


(204, 5)


,location_id,country,country_txt,region,region_txt
0,1,58,Dominican Republic,2,Central America & Caribbean
1,2,130,Mexico,1,North America
2,3,160,Philippines,5,Southeast Asia
3,4,78,Greece,8,Western Europe
4,5,101,Japan,4,East Asia


### 6.2 Fact Table 2

In [9]:
# Merge location_id into fact_incident
fact_incident = fact_incident.merge(
    dim_location,
    on=['country', 'country_txt', 'region', 'region_txt'],
    how='left'
)

# Validate mapping
print("Missing location_id:", fact_incident['location_id'].isna().sum())

fact_incident.head()


Missing location_id: 0


,eventid,iyear,imonth,iday,country,country_txt,region,region_txt,success,suicide,doubtterr,multiple,extended,location_id
0,197000000001,1970,7,2,58,Dominican Republic,2,Central America & Caribbean,1,0,0,0.0,0,1
1,197000000002,1970,0,0,130,Mexico,1,North America,1,0,0,0.0,0,2
2,197001000001,1970,1,0,160,Philippines,5,Southeast Asia,1,0,0,0.0,0,3
3,197001000002,1970,1,0,78,Greece,8,Western Europe,1,0,0,0.0,0,4
4,197001000003,1970,1,0,101,Japan,4,East Asia,1,0,-9,0.0,0,5


In [10]:
fact_incident = fact_incident.drop(
    columns=['country', 'country_txt', 'region', 'region_txt']
)


In [11]:
fact_incident.head()

,eventid,iyear,imonth,iday,success,suicide,doubtterr,multiple,extended,location_id
0,197000000001,1970,7,2,1,0,0,0.0,0,1
1,197000000002,1970,0,0,1,0,0,0.0,0,2
2,197001000001,1970,1,0,1,0,0,0.0,0,3
3,197001000002,1970,1,0,1,0,0,0.0,0,4
4,197001000003,1970,1,0,1,0,-9,0.0,0,5


In [12]:
fact_incident.to_csv(
    "/kaggle/working/fact_incident.csv",
    index=False
)


In [13]:
dim_location.to_csv(
    "/kaggle/working/dim_location.csv",
    index=False
)


### 6.3 Fact Table 3

In [14]:
fact_impact = df_gtd[
    ['eventid', 'nkill', 'nwound', 'property', 'propextent']
].copy()

print(fact_impact.shape)
fact_impact.head()


(209706, 5)


,eventid,nkill,nwound,property,propextent
0,197000000001,1.0,0.0,0,NaN
1,197000000002,0.0,0.0,0,NaN
2,197001000001,1.0,0.0,0,NaN
3,197001000002,NaN,NaN,1,NaN
4,197001000003,NaN,NaN,1,NaN


In [15]:
fact_impact['eventid'].is_unique


True

In [16]:
fact_impact.to_csv(
    "/kaggle/working/fact_impact.csv",
    index=False
)


### 6.4 Fact Table 4

In [17]:
fact_ransom_kidnap = df_gtd[
    [
        'eventid',
        'ishostkid',
        'nhostkid',
        'ransom',
        'ransomamt',
        'ransompaid'
    ]
].copy()

print(fact_ransom_kidnap.shape)
fact_ransom_kidnap.head()


(209706, 6)


,eventid,ishostkid,nhostkid,ransom,ransomamt,ransompaid
0,197000000001,0.0,NaN,0.0,NaN,NaN
1,197000000002,1.0,1.0,1.0,800000.0,NaN
2,197001000001,0.0,NaN,0.0,NaN,NaN
3,197001000002,0.0,NaN,0.0,NaN,NaN
4,197001000003,0.0,NaN,0.0,NaN,NaN


In [18]:
fact_ransom_kidnap['eventid'].is_unique


True

In [19]:
fact_ransom_kidnap.to_csv(
    "/kaggle/working/fact_ransom_kidnap.csv",
    index=False
)


### 6.5 Fact Table 5

In [20]:
fact_flags = df_gtd[
    [
        'eventid',
        'success',
        'suicide',
        'doubtterr',
        'multiple',
        'extended'
    ]
].copy()

print(fact_flags.shape)
fact_flags.head()


(209706, 6)


,eventid,success,suicide,doubtterr,multiple,extended
0,197000000001,1,0,0,0.0,0
1,197000000002,1,0,0,0.0,0
2,197001000001,1,0,0,0.0,0
3,197001000002,1,0,0,0.0,0
4,197001000003,1,0,-9,0.0,0


In [21]:
fact_flags['eventid'].is_unique


True

In [22]:
fact_flags.to_csv(
    "/kaggle/working/fact_flags.csv",
    index=False
)


### 7.2 Dimension Table 2

In [23]:
target_cols = [
    ('targtype1', 'targtype1_txt'),
    ('targtype2', 'targtype2_txt'),
    ('targtype3', 'targtype3_txt')
]

target_frames = []

for id_col, txt_col in target_cols:
    temp = df_gtd[[id_col, txt_col]].dropna()
    temp.columns = ['targettype_id', 'targettype_txt']
    target_frames.append(temp)

dim_target_type = (
    pd.concat(target_frames)
      .drop_duplicates()
      .reset_index(drop=True)
)

print(dim_target_type.shape)
dim_target_type.head()


(22, 2)


,targettype_id,targettype_txt
0,14.0,Private Citizens & Property
1,7.0,Government (Diplomatic)
2,10.0,Journalists & Media
3,3.0,Police
4,21.0,Utilities


In [24]:
dim_target_type['targettype_id'].is_unique


True

In [25]:
dim_target_type.to_csv(
    "/kaggle/working/dim_target_type.csv",
    index=False
)


### 8.1 Bridge Table 1 

In [26]:
bridge_frames = []

for id_col, _ in target_cols:
    temp = df_gtd[['eventid', id_col]].dropna()
    temp.columns = ['eventid', 'targettype_id']
    bridge_frames.append(temp)

bridge_event_target = (
    pd.concat(bridge_frames)
      .drop_duplicates()
      .reset_index(drop=True)
)

print(bridge_event_target.shape)
bridge_event_target.head()


(222906, 2)


,eventid,targettype_id
0,197000000001,14.0
1,197000000002,7.0
2,197001000001,10.0
3,197001000002,7.0
4,197001000003,7.0


In [27]:
bridge_event_target['eventid'].isin(fact_incident['eventid']).all()


np.True_

In [28]:
bridge_event_target['targettype_id'].isin(dim_target_type['targettype_id']).all()


np.True_

In [29]:
bridge_event_target.to_csv(
    "/kaggle/working/bridge_event_target.csv",
    index=False
)


### 7.3 Dimension Table 3

In [30]:
weapon_cols = [
    ('weaptype1', 'weaptype1_txt'),
    ('weaptype2', 'weaptype2_txt'),
    ('weaptype3', 'weaptype3_txt'),
    ('weaptype4', 'weaptype4_txt')
]

weapon_frames = []

for id_col, txt_col in weapon_cols:
    temp = df_gtd[[id_col, txt_col]].dropna()
    temp.columns = ['weapontype_id', 'weapontype_txt']
    weapon_frames.append(temp)

dim_weapon_type = (
    pd.concat(weapon_frames)
      .drop_duplicates()
      .reset_index(drop=True)
)

print(dim_weapon_type.shape)
dim_weapon_type.head()


(12, 2)


,weapontype_id,weapontype_txt
0,13.0,Unknown
1,6.0,Explosives
2,8.0,Incendiary
3,5.0,Firearms
4,2.0,Chemical


In [31]:
dim_weapon_type['weapontype_id'].is_unique


True

In [32]:
dim_weapon_type.to_csv(
    "/kaggle/working/dim_weapon_type.csv",
    index=False
)


### 8.2 Bridge Table 2 

In [33]:
bridge_frames = []

for id_col, _ in weapon_cols:
    temp = df_gtd[['eventid', id_col]].dropna()
    temp.columns = ['eventid', 'weapontype_id']
    bridge_frames.append(temp)

bridge_event_weapon = (
    pd.concat(bridge_frames)
      .drop_duplicates()
      .reset_index(drop=True)
)

print(bridge_event_weapon.shape)
bridge_event_weapon.head()


(224925, 2)


,eventid,weapontype_id
0,197000000001,13.0
1,197000000002,13.0
2,197001000001,13.0
3,197001000002,6.0
4,197001000003,8.0


In [34]:
bridge_event_weapon['eventid'].isin(fact_incident['eventid']).all()


np.True_

In [35]:
bridge_event_weapon['weapontype_id'].isin(dim_weapon_type['weapontype_id']).all()


np.True_

In [36]:
bridge_event_weapon.to_csv(
    "/kaggle/working/bridge_event_weapon.csv",
    index=False
)


### 7.4 Dimension Table 4

In [37]:
attack_cols = [
    ('attacktype1', 'attacktype1_txt'),
    ('attacktype2', 'attacktype2_txt'),
    ('attacktype3', 'attacktype3_txt')
]

attack_frames = []

for id_col, txt_col in attack_cols:
    temp = df_gtd[[id_col, txt_col]].dropna()
    temp.columns = ['attacktype_id', 'attacktype_txt']
    attack_frames.append(temp)

dim_attack_type = (
    pd.concat(attack_frames)
      .drop_duplicates()
      .reset_index(drop=True)
)

print(dim_attack_type.shape)
dim_attack_type.head()


(9, 2)


,attacktype_id,attacktype_txt
0,1.0,Assassination
1,6.0,Hostage Taking (Kidnapping)
2,3.0,Bombing/Explosion
3,7.0,Facility/Infrastructure Attack
4,2.0,Armed Assault


In [38]:
dim_attack_type['attacktype_id'].is_unique


True

In [39]:
dim_attack_type.to_csv(
    "/kaggle/working/dim_attack_type.csv",
    index=False
)


### 8.3 Bridge Table 3 

In [40]:
bridge_frames = []

for id_col, _ in attack_cols:
    temp = df_gtd[['eventid', id_col]].dropna()
    temp.columns = ['eventid', 'attacktype_id']
    bridge_frames.append(temp)

bridge_event_attack = (
    pd.concat(bridge_frames)
      .drop_duplicates()
      .reset_index(drop=True)
)

print(bridge_event_attack.shape)
bridge_event_attack.head()


(218758, 2)


,eventid,attacktype_id
0,197000000001,1.0
1,197000000002,6.0
2,197001000001,1.0
3,197001000002,3.0
4,197001000003,7.0


In [41]:
bridge_event_attack['eventid'].isin(fact_incident['eventid']).all()


np.True_

In [42]:
bridge_event_attack['attacktype_id'].isin(dim_attack_type['attacktype_id']).all()


np.True_

In [44]:
bridge_event_attack.to_csv(
    "/kaggle/working/bridge_event_attack.csv",
    index=False
)


## 10. Summary and Next Steps

This notebook normalized the raw GTD dataset into
fact, dimension, and bridge tables suitable for
analytical use.

The next notebook focuses on:
- Cleaning and validating the modeled tables
- Handling missing values and inconsistencies
- Final preparation for Tableau dashboards
